In [ ]:
from PIL import Image
import os
import numpy as np
from matplotlib import image as mpimg
import matplotlib.pyplot as plt

# Define folder path
folder_number = 'C0001'
folder_path = os.path.join(r"C:\Users\邱宇辰\OneDrive - Universiteit Utrecht\桌面\Utrecht studying\Team challenge\Aneurysm_TC_data\results", folder_number + "saved_images")

# Define color ranges (in RGB format)
red_color = np.array([255, 0, 0])
green_color = np.array([0, 128, 0])
blue_color = np.array([0, 0, 255])
pink_color = np.array([255, 192, 203])  # RGB value for pink color (overlap region)

# Define color tolerance
tolerance = 50

def count_pixels_by_color(image, target_color, tolerance):
    """Use NumPy to count the number of pixels of a specified color in an image"""
    image_array = np.array(image)  # Convert the image to a NumPy array
    color_diff = np.abs(image_array - target_color)  # Calculate color differences
    within_tolerance = np.all(color_diff <= tolerance, axis=-1)  # Check if within tolerance range
    return np.sum(within_tolerance)  # Return the number of pixels that match

def process_images(folder_path, result_file):
    """Process images in the folder and write results to the result file."""
    no_overlap_files = [f for f in os.listdir(folder_path) if '2d_projection_no_overlap_' in f and f.endswith('.png')]
    overlap_files = [f for f in os.listdir(folder_path) if '2d_projection_overlap_' in f and f.endswith('.png')]
    
    no_overlap_dict = {f.split('_')[-1].replace('.png', ''): f for f in no_overlap_files}
    overlap_dict = {f.split('_')[-1].replace('.png', ''): f for f in overlap_files}

    min_overlap_pixels_list = []
    overlap_aneurysm_candidates = []
    overlap_neck_candidates = []

    for angle, no_overlap_filename in no_overlap_dict.items():
        if angle in overlap_dict:
            overlap_filename = overlap_dict[angle]
            
            no_overlap_image = Image.open(os.path.join(folder_path, no_overlap_filename)).convert('RGB')
            overlap_image = Image.open(os.path.join(folder_path, overlap_filename)).convert('RGB')
            
            blood_vessel_pixels = count_pixels_by_color(no_overlap_image, red_color, tolerance)
            connection_region_pixels = count_pixels_by_color(no_overlap_image, green_color, tolerance)
            aneurysm_pixels = count_pixels_by_color(no_overlap_image, blue_color, tolerance)
            pink_overlap_pixels = count_pixels_by_color(overlap_image, pink_color, tolerance)
            
            result_file.write(f"File (no_overlap): {no_overlap_filename}\n")
            result_file.write(f"File (overlap): {overlap_filename}\n")
            result_file.write(f"blood_vessel_pixels: {blood_vessel_pixels}\n")
            result_file.write(f"connection_region_pixels: {connection_region_pixels}\n")
            result_file.write(f"aneurysm_pixels: {aneurysm_pixels}\n")
            result_file.write(f"pink_overlap_pixels: {pink_overlap_pixels}\n")
            result_file.write("-" * 30 + "\n")

            min_overlap_pixels_list.append((pink_overlap_pixels, no_overlap_filename, overlap_filename))

    min_overlap_pixels_list.sort(key=lambda x: x[0])
    top_5_min_overlap = min_overlap_pixels_list[:5]

    for pink_overlap_pixels, no_overlap_filename, overlap_filename in top_5_min_overlap:
        no_overlap_image = Image.open(os.path.join(folder_path, no_overlap_filename)).convert('RGB')
        overlap_image = Image.open(os.path.join(folder_path, overlap_filename)).convert('RGB')
        
        blood_vessel_pixels = count_pixels_by_color(no_overlap_image, red_color, tolerance)
        connection_region_pixels = count_pixels_by_color(no_overlap_image, green_color, tolerance)
        aneurysm_pixels = count_pixels_by_color(no_overlap_image, blue_color, tolerance)
        pink_overlap_pixels = count_pixels_by_color(overlap_image, pink_color, tolerance)
        
        score_aneurysm = aneurysm_pixels
        score_neck = connection_region_pixels
        
        overlap_aneurysm_candidates.append((score_aneurysm, no_overlap_filename, overlap_filename))
        overlap_neck_candidates.append((score_neck, no_overlap_filename, overlap_filename))

    best_overlap_aneurysm = max(overlap_aneurysm_candidates, key=lambda x: x[0]) if overlap_aneurysm_candidates else None
    best_overlap_neck = min(overlap_neck_candidates, key=lambda x: x[0]) if overlap_neck_candidates else None

    return top_5_min_overlap, best_overlap_aneurysm, best_overlap_neck

def save_and_display_image(image, title, save_path, filename_suffix, save_optimal_path):
    """Save and display the image."""
    plt.figure(figsize=(10, 5))
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    save_filename = os.path.basename(save_path).replace('.png', f'_{filename_suffix}.png')
    save_full_path = os.path.join(save_optimal_path, save_filename)
    print(f"Saving {title} to: {save_full_path}")
    plt.savefig(save_full_path, bbox_inches='tight', pad_inches=0, dpi=700)
    print(f"Saved {title} successfully.")
    plt.show()

def main():
    save_optimal_path = os.path.join(folder_path, "optimal")
    if not os.path.exists(save_optimal_path):
        os.makedirs(save_optimal_path)

    with open(os.path.join(folder_path, 'results.txt'), 'w') as result_file:
        top_5_min_overlap, best_overlap_aneurysm, best_overlap_neck = process_images(folder_path, result_file)

    print("\nResults:")
    print(f"Top 5 images with smallest overlap:")
    for i, (pink_overlap_pixels, no_overlap_filename, overlap_filename) in enumerate(top_5_min_overlap, start=1):
        print(f"{i}. {overlap_filename} (Overlap: {pink_overlap_pixels} pixels)")

    if best_overlap_aneurysm:
        print(f"\nImage with best overlap_aneurysm_score: {best_overlap_aneurysm[2]} (Aneurysm Pixels: {best_overlap_aneurysm[0]})")
    else:
        print("No image with best overlap_aneurysm_score found.")

    if best_overlap_neck:
        print(f"\nImage with best overlap_neck_score: {best_overlap_neck[2]} (Connection Region Pixels: {best_overlap_neck[0]})")
    else:
        print("No image with best overlap_neck_score found.")

    # Save and display images
    if top_5_min_overlap:
        min_overlap_pixels_path = os.path.join(folder_path, top_5_min_overlap[0][2])
        min_overlap_pixels = mpimg.imread(min_overlap_pixels_path)
        save_and_display_image(min_overlap_pixels, "Min Overlap Pixels", min_overlap_pixels_path, "min_overlap", save_optimal_path)

    if best_overlap_aneurysm:
        best_overlap_aneurysm_path = os.path.join(folder_path, best_overlap_aneurysm[2])
        best_overlap_aneurysm_image = mpimg.imread(best_overlap_aneurysm_path)
        save_and_display_image(best_overlap_aneurysm_image, "Best Overlap Aneurysm Score", best_overlap_aneurysm_path, "best_overlap_aneurysm", save_optimal_path)

    if best_overlap_neck:
        best_overlap_neck_path = os.path.join(folder_path, best_overlap_neck[2])
        best_overlap_neck_image = mpimg.imread(best_overlap_neck_path)
        save_and_display_image(best_overlap_neck_image, "Best Overlap Neck Score", best_overlap_neck_path, "best_overlap_neck", save_optimal_path)

if __name__ == "__main__":
    main()

Used to check the RGB values of different colors in the image to fix errors. (Previously, there was a problem in the RGB value, this code is used to fix problem )

In [ ]:
from PIL import Image
import numpy as np

# Set the image path
image_path = r"C:\Users\邱宇辰\OneDrive - Universiteit Utrecht\桌面\Utrecht studying\Team challenge\Aneurysm_TC_data\saved_images\2d_projection_overlap_240.png"

# Read the image and convert it to a NumPy array
img = Image.open(image_path).convert("RGB")
img_array = np.array(img)

# Get all unique colors and their pixel counts
unique_colors, counts = np.unique(img_array.reshape(-1, 3), axis=0, return_counts=True)

# Sort by pixel count in descending order
sorted_indices = np.argsort(counts)[::-1]

# Output the top four colors and their corresponding pixel counts
print("Top four color values and their pixel counts in the image:")
for i in range(min(5, len(unique_colors))):
    color = unique_colors[sorted_indices[i]]
    count = counts[sorted_indices[i]]
    print(f"Color {tuple(color)}: {count} pixels")



Top four color values and their pixel counts in the image:
Color (255, 255, 255): 29344207 pixels
Color (255, 0, 0): 1815183 pixels
Color (0, 0, 255): 97940 pixels
Color (255, 192, 203): 67296 pixels
Color (0, 128, 0): 8479 pixels
